In [22]:
import tensorflow as tf

# 1 Incetion模块

In [23]:
class Inception(tf.keras.layers.Layer):
    def __init__(self, c1, c2, c3, c4):
        super().__init__()

        #线路1
        self.p1_1 = tf.keras.layers.Conv2D(c1, kernel_size = 1, padding = 'same', activation = 'relu')
        #线路2
        self.p2_1 = tf.keras.layers.Conv2D(c2[0], kernel_size = 1, padding = 'same', activation = 'relu')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1], kernel_size = 3, padding = 'same', activation = 'relu')
        # 线路3
        self.p3_1 = tf.keras.layers.Conv2D(
            c3[0], kernel_size = 1, activation = 'relu', padding = 'same'
        )
        self.p3_2 = tf.keras.layers.Conv2D(
            c3[1], kernel_size = 5, activation = 'relu', padding = 'same'
        )
        # 线路4
        self.p4_1 = tf.keras.layers.MaxPool2D(
            pool_size = 3, padding = 'same', strides = 1
        )
        self.p4_2 = tf.keras.layers.Conv2D(
            c4, kernel_size = 1, activation = 'relu', padding = 'same'
        )
    
    def call(self, x):
        p1 = self.p1_1(x)

        p2 = self.p2_2(self.p2_1(x))

        p3 = self.p3_2(self.p3_1(x))

        p4 = self.p4_2(self.p4_1(x))

        # 在通道维上concat输出
        outputs = tf.concat([p1, p2, p3, p4], axis=-1)
        return outputs

In [24]:
Inception(64, (96, 128), (16, 32), 32)

<Inception name=inception_10, built=False>

# 2 GoogLeNet构建

## (1)B1模块

In [25]:
# 定义模型输入
inputs = tf.keras.Input(shape = (224, 224, 3), name = 'input')

x = tf.keras.layers.Conv2D(64, kernel_size = 7, strides = 2, padding = 'same', activation = 'relu')(inputs)

x = tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)

## (2)B2模块

In [26]:
x = tf.keras.layers.Conv2D(64, kernel_size = 1, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Conv2D(192, kernel_size = 3, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)

## (3)B3模块

In [27]:
x = Inception(64, (96, 128), (16, 32), 32)(x)
x = Inception(128, (128, 192), (32, 96), 64)(x)
x = tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)

## (4)B4模块

### 辅助分类器

In [28]:
def aux_classifier(x, filter_size):
    # x是输入数据，filter_size：卷积层卷积核个数，全连接层神经元个数
    x = tf.keras.layers.AveragePooling2D(
        pool_size = 5, strides = 3, padding = 'same'
    )(x)
    x = tf.keras.layers.Conv2D(filter_size[0], kernel_size = 1, strides = 1, padding = 'valid', activation = 'relu')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(units = filter_size[1], activation = 'relu')(x)
    x = tf.keras.layers.Dense(units = 10, activation = 'softmax')(x)
    return x

### B4模块

In [29]:
# Inception
x = Inception(192, (96, 208), (16, 48), 64)(x)
# 辅助输出1
aux_output_1 = aux_classifier(x, [128, 1024])
# Inception
x = Inception(160, (112, 224), (24, 64), 64)(x)
# Inception
x = Inception(128, (128, 256), (24, 64), 64)(x)
# Inception
x = Inception(112, (144, 228), (32, 64), 64)(x)
# 辅助输出2
aux_output_2 = aux_classifier(x, [128, 1024])
# Inception
x = Inception(256, (160, 320), (32, 128), 128)(x)
# 池化层
x = tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)

## (5)B5模块

In [30]:
x = Inception(256, (160, 320), (32, 128), 128)(x)
x = Inception(384, (192, 384), (48, 128), 128)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# 输出层
main_output = tf.keras.layers.Dense(10, activation = 'softmax')(x)

## 最后构建GoogLeNet 查看模型结构

In [31]:
model = tf.keras.Model(inputs = inputs, outputs = [main_output, aux_output_1, aux_output_2])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_71 (Conv2D)  │ (None, 112, 112,  │      9,472 │ input[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_15    │ (None, 56, 56,    │          0 │ conv2d_71[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_72 (Conv2D)  │ (None, 56, 56,    │      4,160 │ max_pooling2d_15… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_73 (Conv2D)  │ (None, 56, 56,    │    110,784 │ conv2d_72[0][0]   │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 28, 28,    │          0 │ conv2d_73[0][0]   │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_11        │ (None, 28, 28,    │    163,696 │ max_pooling2d_16… │
│ (Inception)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_12        │ (None, 28, 28,    │    388,736 │ inception_11[0][… │
│ (Inception)         │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_19    │ (None, 14, 14,    │          0 │ inception_12[0][… │
│ (MaxPooling2D)      │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_13        │ (None, 14, 14,    │    376,176 │ max_pooling2d_19… │
│ (Inception)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_14        │ (None, 14, 14,    │    449,160 │ inception_13[0][… │
│ (Inception)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_15        │ (None, 14, 14,    │    510,104 │ inception_14[0][… │
│ (Inception)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_16        │ (None, 14, 14,    │    527,556 │ inception_15[0][… │
│ (Inception)         │ 468)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_17        │ (None, 14, 14,    │    833,792 │ inception_16[0][… │
│ (Inception)         │ 832)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_25    │ (None, 7, 7, 832) │          0 │ inception_17[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_2 │ (None, 5, 5, 512) │          0 │ inception_13[0][… │
│ (AveragePooling2D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_3 │ (None, 5, 5, 468) │          0 │ inception_16[0][

 Total params: 12,573,266 (47.96 MB)

 Trainable params: 12,573,266 (47.96 MB)

 Non-trainable params: 0 (0.00 B)

# 3 数据读取

In [32]:
import numpy as np
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# N H W C
train_images = np.reshape(train_images, (train_images.shape[0], train_images.shape[1], train_images.shape[2], 1))

test_images = np.reshape(test_images, (test_images.shape[0], test_images.shape[1], test_images.shape[2], 1))

In [33]:
# 定义两个方法随机抽取部分样本演示

def get_train(size):
    index = np.random.randint(0, np.shape(train_images)[0], size)

    resize_images = tf.image.resize_with_pad(train_images[index], 224, 224, )

    return resize_images.numpy(), train_labels[index]

def get_test(size):
    index = np.random.randint(0, np.shape(test_images)[0], size)

    resize_images = tf.image.resize_with_pad(test_images[index], 224, 224, )

    return resize_images.numpy(), test_labels[index]

In [34]:
# 获取训练样本和测试样本
train_image, train_label = get_train(256)
test_image, test_label = get_test(128)

# 4 模型编译

- 有三个输出，需指定权重

In [38]:
# 优化器，损失函数，评价指标
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01),
            loss = tf.keras.losses.sparse_categorical_crossentropy,
            metrics = ['accuracy'])

# 5 模型训练

In [39]:
model.fit(train_images, train_labels, batch_size = 128, epochs = 3, verbose = 1, validation_split = 0.1)

Epoch 1/3


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 28, 28)

# 6 模型评估

In [40]:
model.evaluate(test_images, test_labels, verbose = 1)

ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 28, 28)